<a href="https://colab.research.google.com/github/hkhong72/big_data/blob/main/221101_5Chapter18_(%EC%98%88%EC%8A%B5)_%EC%96%B4%ED%85%90%EC%85%98%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import imdb # 영화 평점 가져오기
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
from attention import Attention
import numpy as np
import matplotlib.pyplot as plt
import os, shutil
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE" # KMP에러 방지

ModuleNotFoundError: No module named 'attention'

In [ ]:
# 데이터 불러오기 + 학습셋과 테스트셋으로 나뉙
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=5000)

In [ ]:
# 단어 수 맞추기
X_train = sequence.pad_sequences(X_train, maxlen=500)
X_test = sequence.pad_sequences(X_test, maxlen=500)

In [ ]:
# 모델 구조 설정
model = models.Sequential()
model.add(layers.Embedding(5000, 100))
model.add(layers.Dropout(0.5))
model.add(layers.LSTM(64, return_sequences=True))
model.add(Attention())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1))
model.add(layers.Activation('sigmoid'))

In [ ]:
# 모델 실행 옵션 설정
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 학습 조기 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# 모델 실행
history = model.fit(X_train, y_train, batch_size=40, epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping_callback])

In [ ]:
# 테스트 정확도 출력
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, y_test)[1]))

In [ ]:
# 학습셋과 테스트셋의 오차
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

In [ ]:
# 그래프로 표현
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c="red", label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c="blue", label='Trainset_loss')

# 그래프에 그리드를 주고 레이블 표시
plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()